In [1]:
import numpy as np
import os
import cv2
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [2]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1,out_channels=32, kernel_size=2,padding=1)
        self.conv2 = nn.Conv2d(in_channels=32,out_channels=64, kernel_size=2,padding=1)
        self.conv3 = nn.Conv2d(in_channels=64,out_channels=128, kernel_size=2,padding=1)
        self.fc1 = nn.Linear(128, 50)
        self.fc2 = nn.Linear(50, 10)
    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x),2))
        x = F.relu(F.max_pool2d(self.conv2(x),2))
        x = F.relu(F.max_pool2d(self.conv3(x),7))
        x = x.view(x.size(0),-1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x
    

In [3]:
net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 32, kernel_size=(2, 2), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(2, 2), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(64, 128, kernel_size=(2, 2), stride=(1, 1), padding=(1, 1))
  (fc1): Linear(in_features=128, out_features=50, bias=True)
  (fc2): Linear(in_features=50, out_features=10, bias=True)
)


In [4]:
net(torch.randn(16,1,28,28)).shape

torch.Size([16, 10])

In [5]:
images=[]
for i in range(10):
    images.append(os.listdir('training/'+str(i)))

In [6]:
mass_of_img=list()
mass_of_labels=list()

In [7]:
for i in range(10):
    for j in range(100):
        img_grey=cv2.imread("training/"+str(i)+"/"+images[i][j],cv2.IMREAD_GRAYSCALE)
        thresh = 128
        img_binary = cv2.threshold(img_grey, thresh, 255, cv2.THRESH_BINARY)[1]
        mass_of_img.append(img_binary)
    for k in range(100):
        mass_of_labels.append(i)
mass_of_imgg=np.array(mass_of_img,dtype=object)

In [8]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=0.001)

In [9]:
for epoch in range(20):
    running_loss = 0.0
    for i in range(len(mass_of_imgg)):
        inputs=torch.Tensor(mass_of_imgg[i].astype(np.float64)).reshape(1,1,28,28)
        labels=torch.tensor([mass_of_labels[i]], dtype=torch.int64)
        outputs = net(inputs)
        
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        # print statistics
        #print(loss.item())
        running_loss += loss.item()
        if i % 15 == 14:    # print every 15 mini-batches
            #print('[%d, %5d] loss: %.3f' %
            #      (epoch + 1, i + 1, running_loss / 15))
            running_loss = 0.0
    print(running_loss)
print('Finished Training')
    

0.0
22.86054563522339
0.02161814086139202
17.65230917930603
0.11731681600213051
6.9240357875823975
10.689756155014038
6.984252631664276
5.390925318002701
9.459354400634766
9.643970906734467
14.078176021575928
15.150752305984497
17.63738751411438
17.19473171234131
16.576518535614014
17.08842158317566
17.173312187194824
16.03010618686676
20.485069513320923
Finished Training


In [10]:
img_grey=cv2.imread("training/0/9441.png",cv2.IMREAD_GRAYSCALE)
thresh = 128
img_binary = cv2.threshold(img_grey, thresh, 255, cv2.THRESH_BINARY)[1]
inputs=torch.Tensor(img_binary.astype(np.float64)).reshape(1,1,28,28)
outputs = net(inputs)
print(outputs)
print(torch.max(outputs))
    
    

tensor([[-1.1378, -1.3422, -1.2801, -1.0512, -0.9236, -0.4866, -0.3318, -0.4228,
         -0.6369, -0.4629]], grad_fn=<AddmmBackward0>)
tensor(-0.3318, grad_fn=<MaxBackward1>)
